In [2]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Sequences


### Recep - Recurrence

$$ W(n) = W(n - 1) + n \Rightarrow \text{Balanced}, W(n)\in O(n^2)$$

$$ W(n) = \sqrt{n} W(\sqrt{n}) + n^2 \Rightarrow \text{Root Domoniated}, W(n)\in O(n^2)$$

$$ W(n) = W(\sqrt{n}) + W(n/2) + n \Rightarrow \text{Root Domoniated}, W(n)\in O(n) $$

$$ W(n) = W(n/2) + W(n/3) + 1 \Rightarrow \text{Leaf Domoniated}, W(n)\in O(n^k), \text{where}~(\log_3^2<k<1)$$

## Sequences

- many useful functions for parallel algorithms


Simple to express by example:

$\langle 10, 20, 40 \rangle$

- We'll spend some time defining it more formally so the semantics are precise.

- We'll then define primitive operations over sequences that can be composed to solve a wide array of problems involving sequences.

<br>

First, a quick refresher of sets, relations, and functions...


## Set

> **set**: collection of distinct objects  

- each element of a set appears exactly once
- set with no elements is empty set: $\{\}$ or $\emptyset$

E.g., Cartesian product of sets $A$ and $B = \{(i,j) : i \in A, j \in B\}$
- " tuples $i$ and $j$ *such that* $i$ is in $A$ and $j$ is in $B$ "

## Relation

> A binary **relation** $R$ from a set $A$ to a set $B$ is a subset of the Cartesian product of $A$ and $B$.  

- $R \subseteq A \times B$
- **domain** of $R$ is the set $\{a : (a,b) \in R\}$
- **range** of $R$ is the set $\{b : (a,b) \in R\}$

## Function

>  A **function** or **mapping** from $A$ to $B$ is a relation $R \subset A \times B$ such that: 

- $|R| = |$domain$(R)|$
- that is, for every $a$ in the domain of $R$, there is only one $b$ in the range of $R$ such that $(a,b) \in R$

$A$ is the **domain** and $B$ is the **co-domain**.

## Sequence

> A **sequence** is a function whose domain is a contiguous set of natural numbers starting at zero.

An $\alpha$ **sequence** is a function from $\mathbb{N}$ to $\alpha$ with domain $\{0, \ldots, n-1\}$ for some $n \in \mathbb{N}$

- $\alpha$ specifies the type of the sequence elements

<br>

E.g., $X$ and $Y$ are equivalent sequences:

$ X = \{(0, $ '$a$'$), (1, $ '$b$'$), (2, $ '$c$'$)\} \equiv \langle $'$a$'$, \: $'$b$'$, \: $'$c$'$\rangle$

$ Y = \{(1, $ '$b$'$), (2, $ '$c$'$), (0, $ '$a$'$)\} \equiv \langle $'$a$'$, \: $'$b$'$, \: $'$c$'$\rangle$

<br>

but $Z$ is not a sequence. why not?

$ Z = \{(0, $ '$a$'$), (2, $ '$c$'$)\} $


> with domain $\{0, \ldots, n-1\}$

<br><br><br>

Next, we'll define a number of functions over sequences and use them to solve problems.

For each, we'll show the mathematical definition, and python code.

## Tabulate

**formal definition**:   
$tabulate \: (f : \: \mathbb{N} \rightarrow \alpha)\: (n :\: \mathbb{N}) : \: \mathbb{S}_\alpha = \langle f(0), f(1), \ldots, f(n-1) \rangle$

$tabulate$ is a function that takes as input:
- another function $f$
- a natural number $n$

and returns a sequence of length $n$ by applying $f$ to each element in $\langle 0, \ldots, n-1 \rangle$
 

In [1]:
def double(n):
        return 2*n
    
def tabulate(f, n):
    return [f(i) for i in range(n)]
   
tabulate(double, 10)


[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

### Each call to f(i) can be done in parallel!

In [2]:
from multiprocessing.pool import ThreadPool

def parallel_tabulate(f, n, nthreads=5):
    with ThreadPool(nthreads) as pool:
        results = []
        # launch all tasks
        for i in range(n): 
            results.append(pool.apply_async(f, [i]))
        # wait for all to finish
        return [r.get() for r in results]
    
list(parallel_tabulate(double, 10))

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

## Map

-  like $tabulate$, but applies $f$ to *elements* of sequence, rather than integers.

**formal definition**: 

$ map \: (f : \alpha \rightarrow \beta)(a : \mathbb{S}_\alpha) : \mathbb{S}_\beta = \{(i, f(x)) : (i, x) \in a\}$

$map$ is a function that takes as input:
- another function $f : \alpha \rightarrow \beta$
- a sequence $a$ of type $\mathbb{S}_\alpha$

and returns a sequence of type $\mathbb{S}_\beta$ with length $n$ by applying $f$ to each element $x \in a$



In [2]:
def my_map(f, a):
    return [f(x) for x in a]

def square(x):
    return x**2
 
my_map(square, [4, 16, 25, 49])


[16, 256, 625, 2401]

## Lambda Calculus 

Consists of expressions $e$ in one of three forms:

1. a **variable**, e.g., $x$
2. a **lambda abstraction**, e.g., $(\lambda \: x \: . \: e)$, where $e$ is a function body.
3. an **application**, written $(e_1, e_2)$ for expressions $e_1$, $e_2$.

<br>

e.g., assume $a = \langle 2, 4, 5, 7\rangle$

$map\: (\mathtt{lambda} \: x \: . \: x^2)\: a \equiv \langle x^2 : x \ \in a \rangle \Rightarrow \langle  4, 16, 25, 49 \rangle$

<a href='https://en.wikipedia.org/wiki/Lambda_calculus'>Source</a>

In [3]:
my_map(lambda x:x**2, [4, 16, 25, 49])

[16, 256, 625, 2401]

In [5]:
# In fact, map is built into python:
list(map(lambda x:x**2, [4, 16, 25, 49]))

[16, 256, 625, 2401]

In [1]:
### Version 1

factorial = lambda i: i if i < 2 else i*factorial(i-1)

### Version 2
def factorial(i):
    if i<2: 
        return i
    else:
        return i*factorial(i-1)
    

## Filter

like $map$, but $f$ is a boolean function, and the returned list contains elements where $f(x)$ is True.


$filter$ is a function that takes as input:
- another function $f : \alpha \rightarrow \mathbb{B}$
- a sequence $a$ of type $\mathbb{S}_\alpha$

and returns a sequence of type $\mathbb{S}_\alpha$ with length $\le n$ by applying $f$ to each element $x \in a$ and retaining only those where $f(x)$ is $\mathtt{True}$.

<br>


e.g., assume $a = \langle 2, 4, 5, 7\rangle$

$filter\: \mathtt{isEven} \: a \equiv \langle x : x \in a \: \vert \: \mathtt{isEven}\: x \rangle \Rightarrow \langle  2, 4\rangle$

In [6]:
def my_filter(f, a):
    return [x for x in a if f(x)]

def isEven(x):
    if x%2==0:
        return True
    else:
        return False
    
my_filter(isEven, [4, 16, 25, 36, 49, 64])

[4, 16, 36, 64]

In [7]:
# like map, this also already exists...
list(filter(lambda x:x%2==0, [4, 16, 25, 36, 49, 64, 81, 100]))

[4, 16, 36, 64, 100]

### Example

Given a sequence $a = [1, 2, 6, 6, 6, 8, 6, 6, 1]$, please count how many 6 are there? What is the longest run?

In [2]:
def key_val(x, key):
    if x==key:
        return 1
    else:
        return 0

key = 6
a = [1, 2, 6, 6, 6, 8, 6, 6, 1]
a1 = my_map(lambda x: key_val(x, key), a)
print(a1)

a2 = my_filter(lambda x: key_val(x, key), a)
print(a2)

[0, 0, 1, 1, 1, 0, 1, 1, 0]
[6, 6, 6, 6, 6]


## Iterate

- Iterate over a sequence and accumulate a result that changes at each step (e.g., "running sum")


$iterate$ is a function that takes as input:
- another function $f : \alpha \times \beta \rightarrow \alpha$
- an initial result $x$
- a sequence $a$ of type $\mathbb{S}_\beta$

and returns a value of type $\alpha$ that is the result of applying $f(x,a)$ to each element of the sequence.


<br>

$iterate \: f \: x \: a =
\begin{cases}
x & \hbox{if} \: |a| = 0\\
iterate \: f \:\: f(x, a[0]) \:\:\: a[1 \ldots |a|-1]& \hbox{otherwise}
\end{cases}
$


e.g.

$iterate \:\: + \:\:\: 0 \:\:\: \langle 2,5,1,6 \rangle \Rightarrow ((((0+2)+5)+1)+6) \Rightarrow 14$

<br>

$f(f(f(x, a[0]), a[1]), a[2])\ldots)$

In [3]:
def iterate(f, x, a):
    """
    Params:
      f.....function to apply
      x.....return when a is empty
      a.....input sequence
    """
    print('iterate: calling %s x=%s a=%s' % (f.__name__, x, a))
    if len(a) == 0:
        return x
    else:
        return iterate(f, f(x, a[0]), a[1:])

def plus(x, y):
    return x + y

iterate(plus, 0, [2,5,1,6])

iterate: calling plus x=0 a=[2, 5, 1, 6]
iterate: calling plus x=2 a=[5, 1, 6]
iterate: calling plus x=7 a=[1, 6]
iterate: calling plus x=8 a=[6]
iterate: calling plus x=14 a=[]


14